In [2]:
# Import findspark and initialize. 
import findspark
findspark.init()

In [3]:
# Import packages
from pyspark.sql import SparkSession
import time

# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

The operation couldn’t be completed. Unable to locate a Java Runtime.
Please visit http://www.java.com for information on installing Java.

/opt/anaconda3/lib/python3.11/site-packages/pyspark/bin/spark-class: line 97: CMD: bad array subscript
head: illegal line count -- -1


PySparkRuntimeError: [JAVA_GATEWAY_EXITED] Java gateway process exited before sending its port number.

In [5]:
# 1. Read in the AWS S3 bucket into a DataFrame.
from pyspark import SparkFiles
url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv"
spark.sparkContext.addFile(url)

home_sales_df = spark.read.csv(SparkFiles.get("home_sales_revised.csv"), sep =",", header=True)
home_sales_df.show()

NameError: name 'spark' is not defined

In [4]:
# 2. Create a temporary view of the DataFrame.
home_sales_df.createOrReplaceTempView('home_sales')

NameError: name 'spark' is not defined

In [ ]:
# 3. What is the average price for a four bedroom house sold per year, rounded to two decimal places?

sql = """
SELECT YEAR(date) AS YEAR, ROUND(AVG(price), 2) AS AVG_PRICE
FROM home_sales
WHERE bedrooms = 4
GROUP BY YEAR
ORDER BY YEAR
"""
spark.sql(sql).show()

In [ ]:
# 4. What is the average price of a home for each year the home was built,
# that have 3 bedrooms and 3 bathrooms, rounded to two decimal places?

sql2 = """
SELECT YEAR(date_built) AS YEAR, ROUND(AVG(price), 2) AS AVG_PRICE
FROM home_sales
WHERE bedrooms = 3 AND bathrooms = 3
GROUP BY YEAR
ORDER BY YEAR
"""
spark.sql(sql2).show()

In [ ]:
# 5. What is the average price of a home for each year the home was built,
# that have 3 bedrooms, 3 bathrooms, with two floors,
# and are greater than or equal to 2,000 square feet, rounded to two decimal places?

sql3 = """
SELECT YEAR(date_built) AS YEAR, ROUND(AVG(price), 2) AS AVG_PRICE
FROM home_sales
WHERE bedrooms = 3 AND bathrooms = 3 AND floors =2 AND sqft_living >= 2000
GROUP BY YEAR
ORDER BY YEAR
"""
spark.sql(sql3).show()

In [1]:
# 6. What is the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000? Order by descending view rating. 
# Although this is a small dataset, determine the run time for this query.
#once data is loaded, run again .:. **run twice**

start_time = time.time()

sql4 = """
SELECT view, ROUND(AVG(price), 2) AS AVG_PRICE
FROM home_sales
GROUP BY view
HAVING AVG_PRICE >= 350000
ORDER BY view
"""
spark.sql(sql4).show()

print("--- %s seconds ---" % (time.time() - start_time))

NameError: name 'time' is not defined

In [ ]:
# 7. Cache the the temporary table home_sales.
spark.sql('cache table home_sales')

In [ ]:
# 8. Check if the table is cached.
spark.catalog.isCached('home_sales')

In [ ]:
# 9. Using the cached data, run the last query above, that calculates 
# the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000. 
# Determine the runtime and compare it to the uncached runtime.

start_time = time.time()

sql4 = """
SELECT view, ROUND(AVG(price), 2) AS AVG_PRICE
FROM home_sales
GROUP BY view
HAVING AVG_PRICE >= 350000
ORDER BY view
"""
spark.sql(sql4).show()

print("--- %s seconds ---" % (time.time() - start_time))

In [ ]:
# 10. Partition by the "date_built" field on the formatted parquet home sales data 
home_sales_df.write.partitionBy('date_built').parquet('par_home_sales',mode='overwrite')

In [ ]:
# 11. Read the formatted parquet data.
par_home_sales_df = spark.read.parquet('par_home_sales')

In [ ]:
# 12. Create a temporary table for the parquet data.
par_home_sales_df.createOrReplaceTempVoew('par_home_sales')

In [ ]:
# 13. Using the parquet DataFrame, run the last query above, that calculates 
# the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000. 
# Determine the runtime and compare it to the cached runtime.

start_time = time.time()

sql5 = """
SELECT view, ROUND(AVG(price), 2) AS AVG_PRICE
FROM par_home_sales
GROUP BY view
HAVING AVG_PRICE >= 350000
ORDER BY view
"""
spark.sql(sql5).show()

print("--- %s seconds ---" % (time.time() - start_time))

In [ ]:
# 14. Uncache the home_sales temporary table.
spark.sql('uncache table home_sales')

In [ ]:
# 15. Check if the home_sales is no longer cached
spark.catalog.isCached('home_sales')